In [1]:
import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, Optional

import numpy as np
import pandas as pd
import torch
import torch.nn as nn 
from transformers import AutoTokenizer, EvalPrediction, GlueDataset, GlueDataTrainingArguments, AutoModel, BertPreTrainedModel, AutoConfig, BertModel
from transformers import GlueDataTrainingArguments 
from transformers import (
    Trainer,
    TrainingArguments,
    glue_compute_metrics,
    glue_tasks_num_labels,
    set_seed,
)

import warnings
warnings.filterwarnings('ignore')

In [2]:
#!git clone https://github.com/huggingface/transformers
#!python transformers/utils/download_glue_data.py --tasks RTE
#!pip install transformers

### From non-pretrained BERT

In [3]:
class SequenceClassificationBERT(nn.Module):
      
    def __init__(self, config, bert_model, weights = [0.5,0.5]):
        super().__init__()
        self.config = config
        self.num_labels = config.num_labels
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.bert = bert_model
        self.weights = weights

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # make sure that all the arguments in the forward() function is used
        # somewhere in the code

        ##### 

        # TODO

        #####

        outputs = self.bert(input_ids=input_ids, 
                               attention_mask=attention_mask, 
                               token_type_ids=token_type_ids,
                               position_ids=position_ids,
                               head_mask=head_mask,
                               inputs_embeds=inputs_embeds, 
                               output_attentions=output_attentions,
                               output_hidden_states=output_hidden_states,
                               return_dict=return_dict) 

        pooled_out = outputs[1] 
        pooled_out = self.dropout(pooled_out)
        #print(pooled_out.shape)
        #h_cls = pooled_out[:, 0]
        #print(h_cls.shape)
        logits = self.classifier(pooled_out)

        # RTE (2 classes)
        loss_fct = nn.CrossEntropyLoss(torch.tensor(self.weights, device='cuda'))
        loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1)) 

        # do not change the lines below, so make sure your code works for the
        # lines below

        output = (logits,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output


In [4]:
def compute_metrics_pretrained(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return glue_compute_metrics(data_args.task_name, preds, p.label_ids)

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions[0], axis=1) #predictions
    return glue_compute_metrics(data_args.task_name, preds, p.label_ids)

In [5]:
def non_pretrained(weights):
    set_seed(42)
    num_labels = glue_tasks_num_labels[data_args.task_name]

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    config = AutoConfig.from_pretrained(model_name, output_hidden_states=False, output_attentions=True)
    bert_model = AutoModel.from_config(config)
    # Loading a model from its configuration file does not load the model weights. 
    # It only affects the model’s configuration. Use from_pretrained() to load the model weights.

    model = SequenceClassificationBERT(config=config, bert_model=bert_model,weights=weights)

    train_dataset = GlueDataset(data_args, tokenizer=tokenizer)
    eval_dataset = GlueDataset(data_args, tokenizer=tokenizer, mode="dev")

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )



    trainer.train()
    trainer.evaluate()
    
    d=trainer.evaluate()
    return d['eval_acc'], d['eval_loss']

### From pretrained BERT

In [6]:
def pretrained(weights):
    set_seed(42)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    config = AutoConfig.from_pretrained(model_name, output_hidden_states=False, output_attentions=True)
    bert_model = AutoModel.from_pretrained(model_name, config)

    train_dataset = GlueDataset(data_args, tokenizer=tokenizer)
    eval_dataset = GlueDataset(data_args, tokenizer=tokenizer, mode="dev")

    model = SequenceClassificationBERT(config=config, bert_model=bert_model, weights=weights)


    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics_pretrained,
    )

    trainer.train()
    d=trainer.evaluate()
    return d['eval_acc'], d['eval_loss']

In [7]:
non_pre_acc=[]
pre_acc=[]
non_pre_loss=[]
pre_loss=[]
for i in [0.1, 0.25, 0.5, 0.75, 1]:
    train = pd.read_csv('RTE/train.tsv', sep='\t')
    dev = pd.read_csv('RTE/dev.tsv', sep='\t')
    test = pd.read_csv('RTE/test.tsv', sep='\t')
    
    df1= train[train.label=='not_entailment'].sample(frac=i, random_state=1).reset_index(drop=True)
    print(train.label.value_counts())
    print(df1.shape)
    df2= train[train.label=='entailment'].reset_index(drop=True)
    frames=[df1,df2]
    df=pd.concat(frames).reset_index(drop=True)
    
    df.to_csv('RTE_NEW/train.tsv', sep='\t',index=False)
    dev.to_csv('RTE_NEW/dev.tsv', sep='\t',index=False)
    test.to_csv('RTE_NEW/test.tsv', sep='\t',index=False)
    
    model_name = "bert-base-uncased"

    data_args = GlueDataTrainingArguments(task_name="rte", data_dir="./RTE_NEW")

    training_args = TrainingArguments(
        logging_steps=50, 
        per_device_train_batch_size=32, 
        per_device_eval_batch_size=64, 
        save_steps=1000,
        evaluation_strategy='epoch',
        output_dir="./models/rte",
        overwrite_output_dir=True,
        do_train=True,
        do_eval=True,
        do_predict=True,
        learning_rate=0.00001,
        num_train_epochs=15,
    )
    weights = torch.tensor([1,1/i])
    acc1, loss1= non_pretrained(weights)
    acc2, loss2= pretrained(weights)
    non_pre_acc.append(acc1)
    pre_acc.append(acc2)
    non_pre_loss.append(loss1)
    pre_loss.append(loss2)
    


entailment        1249
not_entailment    1240
Name: label, dtype: int64
(124, 4)


Epoch,Training Loss,Validation Loss,Acc,Runtime,Samples Per Second
1,0.480200,0.384969,0.472924,3.501600,79.107000
2,0.478600,0.367571,0.472924,3.535800,78.342000
3,0.469500,0.339333,0.472924,3.540100,78.246000
4,0.469500,0.343571,0.472924,3.517400,78.752000
5,0.472200,0.345816,0.472924,3.510100,78.916000
6,0.471700,0.332142,0.472924,3.545200,78.133000
7,0.451800,0.334457,0.472924,3.511700,78.878000
8,0.461100,0.435886,0.480144,3.559700,77.815000
9,0.392200,0.457539,0.494585,3.532900,78.406000
10,0.364000,0.805197,0.498195,3.509400,78.930000


Epoch,Training Loss,Validation Loss,Acc,Runtime,Samples Per Second
1,0.552700,0.367333,0.472924,0.734000,377.378000
2,0.450300,0.334950,0.472924,0.736100,376.292000
3,0.405100,0.398969,0.501805,0.734900,376.916000
4,0.342500,0.506724,0.638989,0.735400,376.644000
5,0.261700,0.558075,0.664260,0.735400,376.689000
6,0.162500,0.750224,0.642599,0.734100,377.321000
7,0.123600,0.828695,0.635379,0.734300,377.222000
8,0.100400,0.962463,0.649819,0.734700,377.002000
9,0.069900,1.120447,0.646209,0.732900,377.947000
10,0.061000,1.284714,0.638989,0.734900,376.900000


entailment        1249
not_entailment    1240
Name: label, dtype: int64
(310, 4)


Epoch,Training Loss,Validation Loss,Acc,Runtime,Samples Per Second
1,0.660700,0.648006,0.472924,3.542600,78.191000
2,0.659900,0.589468,0.472924,3.539400,78.262000
3,0.650000,0.545528,0.472924,3.544600,78.148000
4,0.646800,0.549448,0.472924,3.541800,78.208000
5,0.654100,0.560168,0.472924,3.547000,78.093000
6,0.641800,0.527236,0.472924,3.526700,78.544000
7,0.633700,0.535244,0.472924,3.520100,78.691000
8,0.595800,0.714726,0.505415,3.505600,79.016000
9,0.525600,0.764575,0.537906,3.544600,78.148000
10,0.480900,0.783615,0.490975,3.543300,78.175000


...

Epoch,Training Loss,Validation Loss,Acc,Runtime,Samples Per Second
1,0.687200,0.872348,0.527076,3.521600,78.656000
2,0.683200,0.767056,0.527076,3.529600,78.478000
3,0.684400,0.726461,0.527076,3.529200,78.487000
4,0.673200,0.719567,0.527076,3.548000,78.072000
5,0.685700,0.736455,0.523466,3.561900,77.767000
6,0.655900,0.690580,0.519856,3.528400,78.506000
7,0.641800,0.755327,0.534296,3.518700,78.721000
8,0.558700,1.091507,0.527076,3.560500,77.797000
9,0.494000,0.895535,0.534296,3.560700,77.794000
10,0.454600,1.003952,0.512635,3.527700,78.521000


Epoch,Training Loss,Validation Loss,Acc,Runtime,Samples Per Second
1,0.681800,0.726977,0.563177,0.734300,377.220000
2,0.624800,0.712586,0.592058,0.734500,377.145000
3,0.572600,0.706739,0.620939,0.735600,376.579000
4,0.491000,0.761384,0.642599,0.735100,376.830000
5,0.384100,0.839095,0.642599,0.734900,376.900000
6,0.229200,0.938746,0.638989,0.735900,376.384000
7,0.191000,0.987098,0.631769,0.733000,377.875000
8,0.151700,1.207218,0.617329,0.733400,377.672000
9,0.111200,1.310975,0.653430,0.734500,377.140000
10,0.087000,1.300988,0.646209,0.733400,377.694000


entailment        1249
not_entailment    1240
Name: label, dtype: int64
(1240, 4)


Epoch,Training Loss,Validation Loss,Acc,Runtime,Samples Per Second
1,0.649100,0.862271,0.527076,3.528000,78.514000
2,0.643100,0.768264,0.527076,3.541300,78.219000
3,0.651300,0.741244,0.527076,3.523900,78.607000
4,0.636700,0.722118,0.527076,3.505900,79.010000
5,0.648800,0.735004,0.519856,3.547700,78.078000
6,0.618900,0.693059,0.537906,3.552500,77.973000
7,0.615600,0.849838,0.519856,3.508200,78.957000
8,0.543800,0.780382,0.559567,3.526500,78.548000
9,0.496800,0.931023,0.530686,3.540400,78.240000
10,0.497200,0.928974,0.527076,3.529100,78.490000


Epoch,Training Loss,Validation Loss,Acc,Runtime,Samples Per Second
1,0.649600,0.725813,0.552347,0.732100,378.341000
2,0.589000,0.715586,0.577617,0.733500,377.643000
3,0.543800,0.704000,0.638989,0.731900,378.460000
4,0.463800,0.756826,0.635379,0.733000,377.925000
5,0.367500,0.848482,0.638989,0.734000,377.390000
6,0.220600,0.902078,0.631769,0.732600,378.105000
7,0.180400,1.025410,0.635379,0.734800,376.968000
8,0.144600,1.170527,0.624549,0.735000,376.851000
9,0.106500,1.250460,0.638989,0.733100,377.842000
10,0.093600,1.253942,0.642599,0.733100,377.834000


In [8]:
print(non_pre_acc)
print(pre_acc)
print(non_pre_loss)
print(pre_loss)

[0.5270758122743683, 0.5379061371841155, 0.5270758122743683, 0.5487364620938628, 0.5595667870036101]
[0.6462093862815884, 0.6425992779783394, 0.6534296028880866, 0.6534296028880866, 0.6534296028880866]
[0.8413060307502747, 1.0962923765182495, 1.2345272302627563, 1.285244345664978, 1.1910253763198853]
[1.5908265113830566, 1.8645741939544678, 1.9019352197647095, 1.6948457956314087, 1.5574308633804321]


In [ ]:
non_pre_acc1=[]
pre_acc1=[]
non_pre_loss1=[]
pre_loss1=[]
for i in [0.1, 0.25, 0.5, 0.75, 1]:
    train = pd.read_csv('RTE/train.tsv', sep='\t')
    dev = pd.read_csv('RTE/dev.tsv', sep='\t')
    test = pd.read_csv('RTE/test.tsv', sep='\t')
    
    df1= train[train.label=='not_entailment'].reset_index(drop=True)
    
    df2= train[train.label=='entailment'].sample(frac=i, random_state=1).reset_index(drop=True)
    print(train.label.value_counts())
    print(df2.shape)
    frames=[df1,df2]
    df=pd.concat(frames).reset_index(drop=True)
    
    df.to_csv('RTE_NEW/train.tsv', sep='\t',index=False)
    dev.to_csv('RTE_NEW/dev.tsv', sep='\t',index=False)
    test.to_csv('RTE_NEW/test.tsv', sep='\t',index=False)
    
    model_name = "bert-base-uncased"

    data_args = GlueDataTrainingArguments(task_name="rte", data_dir="./RTE_NEW")

    training_args = TrainingArguments(
        logging_steps=50, 
        per_device_train_batch_size=32, 
        per_device_eval_batch_size=64, 
        save_steps=1000,
        evaluation_strategy='epoch',
        output_dir="./models/rte",
        overwrite_output_dir=True,
        do_train=True,
        do_eval=True,
        do_predict=True,
        learning_rate=0.00001,
        num_train_epochs=15,
    )
    weights = torch.tensor([1/i,1])
    acc1, loss1= non_pretrained(weights)
    acc2, loss2= pretrained(weights)
    non_pre_acc1.append(acc1)
    pre_acc1.append(acc2)
    non_pre_loss1.append(loss1)
    pre_loss1.append(loss2)
    


entailment        1249
not_entailment    1240
Name: label, dtype: int64
(125, 4)


Epoch,Training Loss,Validation Loss,Acc,Runtime,Samples Per Second
1,0.208500,0.308046,0.527076,3.526100,78.557000
2,0.191400,0.307737,0.527076,3.501600,79.106000
3,0.202900,0.307088,0.527076,3.538400,78.283000
4,0.194900,0.301927,0.527076,3.537900,78.295000
5,0.200600,0.301159,0.527076,3.502100,79.095000
6,0.194800,0.306284,0.527076,3.505300,79.022000
7,0.202500,0.298461,0.527076,3.541800,78.208000
8,0.187000,0.311382,0.527076,3.502500,79.087000
9,0.191200,0.294798,0.527076,3.540100,78.247000
10,0.177800,0.326065,0.530686,3.502900,79.076000


Epoch,Training Loss,Validation Loss,Acc,Runtime,Samples Per Second
1,0.286400,0.304162,0.527076,0.733000,377.892000
2,0.184700,0.296987,0.527076,1.578500,175.482000
3,0.184700,0.276779,0.552347,0.735800,376.466000
4,0.154100,0.291632,0.570397,0.734900,376.931000
5,0.135500,0.318998,0.570397,0.735800,376.445000
6,0.095700,0.338488,0.599278,0.735800,376.437000
7,0.085100,0.363328,0.602888,0.735300,376.695000
8,0.065600,0.367842,0.606498,0.735200,376.750000
9,0.055800,0.406682,0.599278,0.734000,377.390000
10,0.049500,0.416673,0.631769,0.735500,376.638000


entailment        1249
not_entailment    1240
Name: label, dtype: int64
(312, 4)


Epoch,Training Loss,Validation Loss,Acc,Runtime,Samples Per Second
1,0.370700,0.536404,0.527076,3.665000,75.579000
2,0.349100,0.523328,0.527076,3.488700,79.400000
3,0.367200,0.516419,0.527076,3.501400,79.111000
4,0.352300,0.510556,0.527076,3.518900,78.717000
5,0.363300,0.509069,0.527076,3.500700,79.127000
6,0.349700,0.519474,0.527076,3.521800,78.653000
7,0.363500,0.489655,0.527076,3.545900,78.118000
8,0.332700,0.512821,0.527076,3.504800,79.034000
9,0.321500,0.578206,0.541516,3.526100,78.556000
10,0.284900,0.635432,0.523466,3.502400,79.089000


In [ ]:
print(non_pre_acc1)
print(pre_acc1)
print(non_pre_loss1)
print(pre_loss1)